In [129]:
!pip install datasets

In [130]:
# all the important libraries from pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

In [131]:
import math

In [132]:
#HuggingFace libraries
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [133]:
#pathlib
from pathlib import Path

In [134]:
#importing library of warnings
import warnings
#Library for progress bars in loops
from tqdm import tqdm
#typing
from typing import Any

In [135]:
# Creating Input Embeddings
class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model  # Dimension of embeddings
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # Multiply by sqrt(d_model) for normalization
        return self.embedding(x) * math.sqrt(self.d_model)


In [136]:
# Creating the Positional Encoding
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        # Create matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)

        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # shape: (1, seq_len, d_model)

        # Register pe as buffer so it moves with model.to(device)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add positional encoding to input
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


In [137]:
# Creating Layer Normalization
class LayerNormalization(nn.Module):

    def __init__(self, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps

        # scale (γ) and shift (β) parameters — learnable
        self.weight = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        return self.weight * (x - mean) / (std + self.eps) + self.bias


In [138]:
# Creating Feed Forward Layers
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.activation = nn.GELU()  # Better than ReLU for transformers
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        return x


In [139]:
# Multi-Head Attention Block
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % h == 0, "d_model must be divisible by number of heads"

        self.d_model = d_model
        self.h = h
        self.d_k = d_model // h

        # Linear layers for q, k, v and final output
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):

        batch_size = q.size(0)

        # Project and split into heads: (B, S, D) -> (B, H, S, d_k)
        def shape(x):
            return x.view(batch_size, -1, self.h, self.d_k).transpose(1, 2)

        query = shape(self.w_q(q))
        key   = shape(self.w_k(k))
        value = shape(self.w_v(v))

        # Scaled Dot-Product Attention
        scores = (query @ key.transpose(-2, -1)) / math.sqrt(self.d_k)

        if mask is not None:
            # ensure mask shape: (B, 1, 1, S) so it broadcasts correctly
            if mask.dim() == 3:
                mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e4)  # stable for FP16

        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        x = attn @ value  # -> (B, H, S, d_k)

        # Combine heads back: (B, H, S, d_k) -> (B, S, D)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        return self.w_o(x)


In [140]:
# Building Residual Connection
class ResidualConnection(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1):
        super().__init__()
        self.norm = LayerNormalization()  # Normalization before sublayer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        # Apply norm → sublayer → dropout then residual add
        return x + self.dropout(sublayer(self.norm(x)))



In [141]:
# Building Encoder Block
class EncoderBlock(nn.Module):

    def __init__(
        self,
        d_model: int,
        self_attention_block: MultiHeadAttentionBlock,
        feed_forward_block: FeedForwardBlock,
        dropout: float
    ):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block

        # Two residual connections: 1 for MHA, 1 for FFN
        self.residual_connections = nn.ModuleList([
            ResidualConnection(d_model, dropout),
            ResidualConnection(d_model, dropout)
        ])

    def forward(self, x, src_mask):
        # Self-attention residual
        x = self.residual_connections[0](
            x, lambda x: self.self_attention_block(x, x, x, src_mask)
        )

        # Feed-forward residual
        x = self.residual_connections[1](x, self.feed_forward_block)

        return x


In [142]:
# Building Encoder
class Encoder(nn.Module):

    def __init__(self, d_model: int, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()  # will operate on last dim (d_model)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


In [143]:
# Building Decoder Block
class DecoderBlock(nn.Module):

    def __init__(
        self,
        d_model: int,
        self_attention_block: MultiHeadAttentionBlock,
        cross_attention_block: MultiHeadAttentionBlock,
        feed_forward_block: FeedForwardBlock,
        dropout: float
    ):
        super().__init__()

        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block

        # 3 Residuals: Self-Attn, Cross-Attn, FeedForward
        self.residual_connections = nn.ModuleList([
            ResidualConnection(d_model, dropout),
            ResidualConnection(d_model, dropout),
            ResidualConnection(d_model, dropout)
        ])

    def forward(self, x, encoder_output, src_mask, tgt_mask):

        # Masked self-attention
        x = self.residual_connections[0](
            x, lambda x: self.self_attention_block(x, x, x, tgt_mask)
        )

        # Cross-attention with encoder output
        x = self.residual_connections[1](
            x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask)
        )

        # Feed-forward processing
        x = self.residual_connections[2](x, self.feed_forward_block)

        return x


In [144]:
# Building Decoder
class Decoder(nn.Module):

    def __init__(self, d_model: int, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()  # Normalize final decoder output

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)


In [145]:
# Building Linear Projection Layer (Decoder Output -> Vocabulary)
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # Convert to float32 for numerical stability before softmax
        return torch.log_softmax(self.proj(x).float(), dim=-1)


In [146]:
# Creating the Transformer Architecture
class Transformer(nn.Module):

    def __init__(
        self,
        encoder: Encoder,
        decoder: Decoder,
        src_embed: InputEmbeddings,
        tgt_embed: InputEmbeddings,
        src_pos: PositionalEncoding,
        tgt_pos: PositionalEncoding,
        projection_layer: ProjectionLayer
    ):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        return self.projection_layer(x)

    # NEW: forward method integrating the pipeline
    def forward(self, src, tgt, src_mask, tgt_mask):
        encoder_output = self.encode(src, src_mask)
        output = self.decode(encoder_output, src_mask, tgt, tgt_mask)
        return self.project(output)


In [147]:
# Building & Initializing Transformer
def build_transformer(
    src_vocab_size: int,
    tgt_vocab_size: int,
    src_seq_len: int,
    tgt_seq_len: int,
    d_model: int = 256,  # optimized for Colab GPU
    N: int = 3,
    h: int = 4,
    dropout: float = 0.1,
    d_ff: int = 512   # optimized for speed
) -> Transformer:

    # Embeddings
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Positional encodings
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Encoder stack
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_blocks.append(
            EncoderBlock(d_model, encoder_self_attention, feed_forward, dropout)
        )

    # Decoder stack
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_blocks.append(
            DecoderBlock(d_model, decoder_self_attention, decoder_cross_attention, feed_forward, dropout)
        )

    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(
        encoder,
        decoder,
        src_embed,
        tgt_embed,
        src_pos,
        tgt_pos,
        projection_layer
    )

    # Xavier initialization for stability
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer


In [148]:
# Defining Tokenizer
def build_tokenizer(config, ds, lang):

    tokenizer_path = Path(config['tokenizer_file'].format(lang))

    # Ensure folder exists
    tokenizer_path.parent.mkdir(parents=True, exist_ok=True)

    # If tokenizer doesn't already exist → create one
    if not tokenizer_path.exists():
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()

        trainer = WordLevelTrainer(
            special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"],
            min_frequency=1   # allow rare words when dataset is small
        )

        # Use dataset iterator
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)

        tokenizer.save(str(tokenizer_path))
        print(f"Trained tokenizer saved for language: {lang}")

    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
        print(f"Loaded existing tokenizer: {tokenizer_path.name}")

    return tokenizer


In [149]:
def get_all_sentences(ds, lang):
    for pair in ds:
        if lang in pair:      # ensure key exists (e.g., "en" or "de")
            yield pair[lang]


In [150]:
def get_ds(config):

    # Load Multi30k dataset
    ds_raw = load_dataset("bentrevett/multi30k", split="train")


    # Tokenizers
    tokenizer_src = build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = build_tokenizer(config, ds_raw, config['lang_tgt'])

    # Split dataset
    split_ds = ds_raw.train_test_split(test_size=0.1, seed=42)
    train_ds_raw = split_ds['train']
    val_ds_raw = split_ds['test']

    # Limit dataset size
    train_limit = min(config["train_subset"], len(train_ds_raw))
    val_limit = min(config["val_subset"], len(val_ds_raw))

    train_ds_raw = train_ds_raw.select(range(train_limit))
    val_ds_raw = val_ds_raw.select(range(val_limit))

    # Create PyTorch datasets
    train_ds = BilingualDataset(
        train_ds_raw, tokenizer_src, tokenizer_tgt,
        config["lang_src"], config["lang_tgt"], config["seq_len"]
    )
    val_ds = BilingualDataset(
        val_ds_raw, tokenizer_src, tokenizer_tgt,
        config["lang_src"], config["lang_tgt"], config["seq_len"]
    )

    # Dataloaders
    train_dataloader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True)
    val_dataloader = DataLoader(val_ds, batch_size=1)

    print(f"Train samples: {len(train_ds_raw)}, Val samples: {len(val_ds_raw)}")

    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt


In [151]:
def causal_mask(size):
    # 1 = allowed, 0 = masked
    mask = torch.tril(torch.ones(size, size)).unsqueeze(0)
    return mask == 1


In [152]:
class BilingualDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = tokenizer_tgt.token_to_id("[SOS]")
        self.eos_token = tokenizer_tgt.token_to_id("[EOS]")
        self.pad_token = tokenizer_tgt.token_to_id("[PAD]")

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        pair = self.ds[idx]
        src_text = pair[self.src_lang]
        tgt_text = pair[self.tgt_lang]

        # Tokenize once only
        enc_tokens = self.tokenizer_src.encode(src_text).ids
        dec_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Trim if too long
        enc_tokens = enc_tokens[:self.seq_len - 2]
        dec_tokens = dec_tokens[:self.seq_len - 1]

        # Padding
        enc_pad_count = self.seq_len - len(enc_tokens) - 2
        dec_pad_count = self.seq_len - len(dec_tokens) - 1

        # Encoder: [SOS] + src + [EOS] + PAD
        encoder_input = torch.tensor(
            [self.sos_token] + enc_tokens + [self.eos_token] +
            [self.pad_token] * enc_pad_count,
            dtype=torch.long
        )

        # Decoder input: [SOS] + tgt + PAD
        decoder_input = torch.tensor(
            [self.sos_token] + dec_tokens +
            [self.pad_token] * dec_pad_count,
            dtype=torch.long
        )

        # Labels: tgt + [EOS] + PAD
        label = torch.tensor(
            dec_tokens + [self.eos_token] +
            [self.pad_token] * dec_pad_count,
            dtype=torch.long
        )

        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        # Masks
        encoder_mask = (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0)
        decoder_mask = (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0) \
                       & causal_mask(decoder_input.size(0))

        return {
            "encoder_input": encoder_input,
            "decoder_input": decoder_input,
            "encoder_mask": encoder_mask,
            "decoder_mask": decoder_mask,
            "label": label,
            "src_text": src_text,
            "tgt_text": tgt_text,
        }


In [153]:
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Encode source
    encoder_output = model.encode(source, source_mask)

    # Decoder starts with <SOS>
    decoder_input = torch.tensor([[sos_idx]], device=device)

    while decoder_input.size(1) < max_len:

        # Causal + padding mask for decode steps
        decoder_mask = causal_mask(decoder_input.size(1)).to(device)

        # Decode
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # Project last token only
        logits = model.project(out[:, -1])
        next_token = torch.argmax(logits, dim=-1)

        # Append token
        decoder_input = torch.cat(
            [decoder_input, next_token.unsqueeze(0)], dim=1
        )

        # Stop if EOS generated
        if next_token.item() == eos_idx:
            break

    return decoder_input.squeeze(0)


In [154]:
def beam_search_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device, beam_size=5):
    sos_idx = tokenizer_tgt.token_to_id("[SOS]")
    eos_idx = tokenizer_tgt.token_to_id("[EOS]")

    encoder_output = model.encode(source, source_mask)

    sequences = [[torch.tensor([sos_idx], device=device), 0.0]]

    for _ in range(max_len):
        all_candidates = []
        for seq, score in sequences:
            if seq[-1].item() == eos_idx:
                all_candidates.append((seq, score))
                continue

            seq_input = seq.unsqueeze(0)
            mask = causal_mask(seq_input.size(1)).to(device)

            with torch.no_grad():
                decoder_output = model.decode(encoder_output, source_mask, seq_input, mask)
                logits = model.project(decoder_output[:, -1])
                probabilities = torch.log_softmax(logits, dim=-1).squeeze(0)

            # Top beam_size candidates
            topk_probs, topk_ids = torch.topk(probabilities, beam_size)

            for prob, idx in zip(topk_probs, topk_ids):
                candidate = torch.cat([seq, idx.unsqueeze(0)])
                all_candidates.append((candidate, score + prob.item()))

        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_size]

    return sequences[0][0]


In [155]:
# Evaluate model on validation dataset
#switched to beam search for better results
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, num_examples=2, beam_size=5):
    model.eval()
    count = 0
    console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1

            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)

            # Validation loader must have batch_size = 1
            assert encoder_input.size(0) == 1

            # Use Beam Search instead of greedy
            model_out = beam_search_decode(
                model, encoder_input, encoder_mask,
                tokenizer_src, tokenizer_tgt,
                max_len, device,
                beam_size=beam_size
            )

            # Decode text
            source_text = batch['src_text'][0]
            target_text = batch['tgt_text'][0]
            model_out_text = tokenizer_tgt.decode(model_out.tolist())

            # Display results
            print_msg("-" * console_width)
            print_msg(f"SOURCE:    {source_text}")
            print_msg(f"TARGET:    {target_text}")
            print_msg(f"PREDICTED: {model_out_text}")

            if count >= num_examples:
                break


In [156]:
def get_model(config, vocab_src_len, vocab_tgt_len, device):
    model = build_transformer(
        vocab_src_len,
        vocab_tgt_len,
        config["seq_len"],
        config["seq_len"],
        d_model=config["d_model"],
        N=config["num_layers"],
        h=config["num_heads"],
        dropout=config["dropout"],
        d_ff=config["d_ff"]
    )

    return model.to(device)


In [157]:
def get_config():
    return {
        "batch_size": 4,
        "num_epochs": 30,
        "lr": 1e-4,

        "seq_len": 80,

        "d_model": 256,
        "num_layers": 3,
        "num_heads": 4,
        "dropout": 0.1,
        "d_ff": 512,

        # Change target language to German
        "lang_src": "en",
        "lang_tgt": "de",

        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": None,
        "tokenizer_file": "tokenizers/tokenizer_{0}.json",
        "experiment_name": "runs/tmodel_en_de",

        "train_subset": 5000,
        "val_subset": 500,
    }


In [158]:
def train_model(config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f" Using device: {device}")

    # Ensure model + tokenizer folders exist
    Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)
    Path("tokenizers").mkdir(parents=True, exist_ok=True)

    # Load dataset + tokenizers
    train_dl, val_dl, tok_src, tok_tgt = get_ds(config)

    # Build model
    model = build_transformer(
        tok_src.get_vocab_size(),
        tok_tgt.get_vocab_size(),
        config["seq_len"],
        config["seq_len"],
        d_model=config["d_model"],
        N=config["num_layers"],
        h=config["num_heads"],
        dropout=config["dropout"],
        d_ff=config["d_ff"],
    ).to(device)

    writer = SummaryWriter(config["experiment_name"])
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    use_amp = (device.type == "cuda")
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    pad_id = tok_tgt.token_to_id("[PAD]")
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id, label_smoothing=0.1)

    initial_epoch = 0
    global_step = 0

    if config["preload"]:
        ckpt = get_weights_file_path(config, config["preload"])
        print(f"Loading checkpoint: {ckpt}")
        state = torch.load(ckpt, map_location=device)
        model.load_state_dict(state["model_state_dict"])
        optimizer.load_state_dict(state["optimizer_state_dict"])
        global_step = state["global_step"]
        initial_epoch = state["epoch"] + 1

    # Training Loop
    for epoch in range(initial_epoch, config["num_epochs"]):
        model.train()
        loop = tqdm(train_dl, desc=f"Epoch {epoch:02d}")

        for batch in loop:
            encoder_input = batch["encoder_input"].to(device)
            decoder_input = batch["decoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)
            decoder_mask = batch["decoder_mask"].to(device)
            label = batch["label"].to(device)

            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=use_amp):
                enc_out = model.encode(encoder_input, encoder_mask)
                dec_out = model.decode(enc_out, encoder_mask, decoder_input, decoder_mask)
                proj = model.project(dec_out)

                loss = loss_fn(
                    proj.view(-1, tok_tgt.get_vocab_size()),
                    label.view(-1)
                )

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            loop.set_postfix(loss=f"{loss.item():.4f}")
            writer.add_scalar("train/loss", loss.item(), global_step)

            global_step += 1

        # Validation
        run_validation(
            model, val_dl,
            tok_src, tok_tgt,
            config["seq_len"],
            device,
            lambda msg: loop.write(msg)
        )

        # Save each epoch
        ckpt_path = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "global_step": global_step
        }, ckpt_path)
        print(f" Saved checkpoint: {ckpt_path}")


In [159]:
def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder']
    model_basename = config['model_basename']
    model_filename = f"{model_basename}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)


In [160]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    torch.cuda.empty_cache()  # optional GPU mem cleanup

    config = get_config()
    train_model(config)


⚡ Using device: cuda
Trained tokenizer saved for language: en
Trained tokenizer saved for language: de
Train samples: 5000, Val samples: 500


Epoch 00: 100%|██████████| 1250/1250 [01:09<00:00, 18.07it/s, loss=6.0254]


--------------------------------------------------------------------------------
SOURCE:    A lady wearing green and white shorts and top is on the beach clapping her hands.
TARGET:    Eine Dame mit grün-weißen Shorts und Oberteil ist auf dem Strand und klatscht in die Hände.
PREDICTED: Ein Mann in einem in einem , einem , und , einem , .
--------------------------------------------------------------------------------
SOURCE:    A couple takes their own picture in front of the Arc De TRiomphe, from across the street.
TARGET:    Ein Paar macht auf der anderen Straßenseite des Arc de Triomphe ein Bild von sich.
PREDICTED: Ein Mann in einem in einem , einem , und , einem , .
💾 Saved checkpoint: weights/tmodel_00.pt


Epoch 01: 100%|██████████| 1250/1250 [01:08<00:00, 18.34it/s, loss=5.0658]


--------------------------------------------------------------------------------
SOURCE:    A lady wearing green and white shorts and top is on the beach clapping her hands.
TARGET:    Eine Dame mit grün-weißen Shorts und Oberteil ist auf dem Strand und klatscht in die Hände.
PREDICTED: Ein Mann in einem Hemd in einem roten roten weißen weißen blauen Straße .
--------------------------------------------------------------------------------
SOURCE:    A couple takes their own picture in front of the Arc De TRiomphe, from across the street.
TARGET:    Ein Paar macht auf der anderen Straßenseite des Arc de Triomphe ein Bild von sich.
PREDICTED: Ein Mann in einem Hemd in einem roten roten weißen blauen Straße .
💾 Saved checkpoint: weights/tmodel_01.pt


...

--------------------------------------------------------------------------------
SOURCE:    A lady wearing green and white shorts and top is on the beach clapping her hands.
TARGET:    Eine Dame mit grün-weißen Shorts und Oberteil ist auf dem Strand und klatscht in die Hände.
PREDICTED: Eine Dame in grünen Shorts und Shorts blickt am Strand entlang .
--------------------------------------------------------------------------------
SOURCE:    A couple takes their own picture in front of the Arc De TRiomphe, from across the street.
TARGET:    Ein Paar macht auf der anderen Straßenseite des Arc de Triomphe ein Bild von sich.
PREDICTED: Ein Paar schneidet das Bild vor der Rückseite eines Autos über die Straße .
💾 Saved checkpoint: weights/tmodel_27.pt


Epoch 28: 100%|██████████| 1250/1250 [01:04<00:00, 19.37it/s, loss=2.7209]


--------------------------------------------------------------------------------
SOURCE:    A lady wearing green and white shorts and top is on the beach clapping her hands.
TARGET:    Eine Dame mit grün-weißen Shorts und Oberteil ist auf dem Strand und klatscht in die Hände.
PREDICTED: Eine Dame in grünen Shorts und Shorts geht am Strand entlang .
--------------------------------------------------------------------------------
SOURCE:    A couple takes their own picture in front of the Arc De TRiomphe, from across the street.
TARGET:    Ein Paar macht auf der anderen Straßenseite des Arc de Triomphe ein Bild von sich.
PREDICTED: Ein Paar küsst sich vor dem Kopf vor dem Rücken eines Autos über die Straße .
💾 Saved checkpoint: weights/tmodel_28.pt


Epoch 29: 100%|██████████| 1250/1250 [01:05<00:00, 18.99it/s, loss=2.1333]


--------------------------------------------------------------------------------
SOURCE:    A lady wearing green and white shorts and top is on the beach clapping her hands.
TARGET:    Eine Dame mit grün-weißen Shorts und Oberteil ist auf dem Strand und klatscht in die Hände.
PREDICTED: Eine Dame mit grünen Shorts und Shorts blickt über die Hände am Strand .
--------------------------------------------------------------------------------
SOURCE:    A couple takes their own picture in front of the Arc De TRiomphe, from across the street.
TARGET:    Ein Paar macht auf der anderen Straßenseite des Arc de Triomphe ein Bild von sich.
PREDICTED: Ein Paar küsst sich vor einem alten Paar vor der Straße .
💾 Saved checkpoint: weights/tmodel_29.pt


In [173]:
# Load config
config = get_config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizers
train_dl, val_dl, tokenizer_src, tokenizer_tgt = get_ds(config)

# Build model
vocab_src_len = tokenizer_src.get_vocab_size()
vocab_tgt_len = tokenizer_tgt.get_vocab_size()
model = get_model(config, vocab_src_len, vocab_tgt_len, device)

# Load latest checkpoint
CHECKPOINT_EPOCH = "29"
checkpoint_path = get_weights_file_path(config, CHECKPOINT_EPOCH)

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
print(f"Loaded model checkpoint: {checkpoint_path}")


Loaded existing tokenizer: tokenizer_en.json
Loaded existing tokenizer: tokenizer_de.json
Train samples: 5000, Val samples: 500
Loaded model checkpoint: weights/tmodel_29.pt


In [174]:
def translate_sentence(model, tokenizer_src, tokenizer_tgt, sentence, device, max_len=80):
    model.eval()

    # Tokenize source sentence
    enc_tokens = tokenizer_src.encode(sentence).ids
    enc_tokens = enc_tokens[:max_len - 2]  # safety clipping
    src = torch.tensor(
        [tokenizer_src.token_to_id("[SOS]")] + enc_tokens + [tokenizer_src.token_to_id("[EOS]")]
    ).unsqueeze(0).to(device)

    # Create encoder mask
    src_mask = (src != tokenizer_src.token_to_id("[PAD]")).unsqueeze(1).unsqueeze(1)

    # Run encoder
    with torch.no_grad():
        enc_out = model.encode(src, src_mask)

    # Greedy decode loop
    tgt = torch.tensor([[tokenizer_tgt.token_to_id("[SOS]")]]).to(device)

    for _ in range(max_len):
        tgt_mask = causal_mask(tgt.size(1)).to(device)
        with torch.no_grad():
            dec_out = model.decode(enc_out, src_mask, tgt, tgt_mask)
            logits = model.project(dec_out[:, -1])
            next_token = torch.argmax(logits, dim=-1)

        tgt = torch.cat([tgt, next_token.unsqueeze(0)], dim=1)

        if next_token.item() == tokenizer_tgt.token_to_id("[EOS]"):
            break

    # Convert token IDs → text
    out_tokens = tgt.squeeze().tolist()
    return tokenizer_tgt.decode(out_tokens)


In [175]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sentence = "the children are playing in the park."
translated = translate_sentence(model, tokenizer_src, tokenizer_tgt, sentence, device)
print("INPUT :", sentence)
print("OUTPUT:", translated)


INPUT : the children are playing in the park.
OUTPUT: Kinder spielen im Park .
